In [13]:
import pandas as pd
import numpy as np
import gc
import re
import MeCab

In [14]:
work_path_lizhi = '/home/mmde-lab/s3/59e875d818c7b/supership/'
y_cate_path = 'paring_table.csv.gz'
df_ycate = pd.read_csv(work_path_lizhi + y_cate_path, compression='gzip')

In [10]:
luxa_click_log_path = 'luxa_click_log.csv.gz'
df_lcl = pd.read_csv(work_path_lizhi + luxa_click_log_path, compression='gzip')

In [18]:
len(df_ycate.vuid.value_counts())

181838

In [2]:
account_path = 'kddi_r_account_full_20180410.tsv.gz'
order_path = 'kddi_r_pay_order.tsv.gz'
deal_path = 'kddi_r_deal.tsv_20180328.gz'
deal_profile_path = 'kddi_r_deal_profile_20180427.tsv.gz'
work_path = '/home/mmde-lab/s3/59e875d818c7b/'
deal_profile_path2 = 'kddi_r_deal_profile_20171222.tsv.gz'
df_account = pd.read_csv(work_path + account_path, sep = '\t', compression='gzip')
df_order = pd.read_csv(work_path + order_path, sep = '\t', compression='gzip')
df_deal = pd.read_csv(work_path + deal_path, sep = '\t', compression='gzip')
df_dealprofile = pd.read_csv(work_path + deal_profile_path, sep = '\t', compression='gzip')
df_dealprofile2 = pd.read_csv(work_path + deal_profile_path2, sep = '\t', compression='gzip')

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9,14,15,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (10,13,26,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,3,9,10,13,17,18,24,25,26,29

In [2]:
df_order.ins_tm.min()

'2016-10-01 00:00:01'

In [6]:
#delete website tag 
pattern = re.compile(r'<(.*?)>')
df_dealprofile.deal_description = df_dealprofile.deal_description.map(lambda x: str(re.sub(pattern, '', str(x))))
df_dealprofile2.deal_description = df_dealprofile2.deal_description.map(lambda x: str(re.sub(pattern, '', str(x))))

In [7]:
df_dealprofile.deal_title = df_dealprofile.deal_title.map(lambda x: str(re.sub(pattern, '', str(x))))
df_dealprofile2.deal_title = df_dealprofile2.deal_title.map(lambda x: str(re.sub(pattern, '', str(x))))

In [8]:
# preprocess deal_description
mecab = MeCab.Tagger('-Ochasen')
def rawdcrp2tokendcrp(rawdescription):
    deal_des = []
    for line in rawdescription:
        tokenlst = mecab.parse(line)
        pattern1 = re.compile(r'(.*?)(名詞|動詞|形容詞)(.*?)')
        new_dcp1 = pattern1.findall(tokenlst)
        pattern2 = re.compile('(.*?)\t')
        sentence_des = []
        for token in range(len(new_dcp1)):
            matchobjdes = re.match(pattern2, new_dcp1[token][0])
            if(matchobjdes != None):
                sentence_des.append(matchobjdes.group(1))
        deal_des.append(sentence_des)
    return deal_des
df_dealprofile.deal_description = df_dealprofile.deal_description.map(lambda x: x.split('。'))
df_dealprofile.deal_description = df_dealprofile.deal_description.map(lambda x: rawdcrp2tokendcrp(x))
df_dealprofile2.deal_description = df_dealprofile2.deal_description.map(lambda x: x.split('。'))
df_dealprofile2.deal_description = df_dealprofile2.deal_description.map(lambda x: rawdcrp2tokendcrp(x)) 

In [9]:
df_dealprofile.deal_title = df_dealprofile.deal_title.map(lambda x: x.split('。'))
df_dealprofile.deal_title = df_dealprofile.deal_title.map(lambda x: rawdcrp2tokendcrp(x)) 
df_dealprofile2.deal_title = df_dealprofile2.deal_title.map(lambda x: x.split('。'))
df_dealprofile2.deal_title = df_dealprofile2.deal_title.map(lambda x: rawdcrp2tokendcrp(x)) 

In [10]:
df_dealprofile.to_csv('df_dealprofile_afterprep201804.csv.gz', compression = 'gzip')
df_dealprofile2.to_csv('df_dealprofile_afterprep201712.csv.gz', compression = 'gzip')

In [11]:
import datetime
df_dealprofile['ins_tm'] = pd.to_datetime(df_dealprofile['ins_tm'],format='%Y-%m-%d %H:%M:%S')
df_dealprofile2['ins_tm'] = pd.to_datetime(df_dealprofile2['ins_tm'],format='%Y-%m-%d %H:%M:%S')
df_dealprofile['date'] = df_dealprofile.ins_tm.dt.date
df_dealprofile2['date'] = df_dealprofile2.ins_tm.dt.date

In [12]:
train_dealprofile = df_dealprofile.loc[df_dealprofile.date <= datetime.date(2017,9,14)]
train_dealprofile2 = df_dealprofile2.loc[df_dealprofile2.date <= datetime.date(2017,9,14)]

In [ ]:
work_path2 = '/home/mmde-lab/lizhi/ss_contents/'
url_path0 = 'crawl_result_domain_0.csv.gz'
url_path1 = 'crawl_result_domain_1.csv.gz'
url_path2 = 'crawl_result_domain_2.csv.gz'
df_url0 = pd.read_csv(work_path2 + url_path0, sep = ',', compression='gzip')
df_url1 = pd.read_csv(work_path2 + url_path1, sep = ',', compression='gzip')
df_url2 = pd.read_csv(work_path2 + url_path2, sep = ',', compression='gzip')
df_url = df_url0.append(df_url1)
df_url = df_url.append(df_url2)
(df_url.description.notnull() & df_url.keywords.notnull()).sum()
(df_url.description.notnull() & df_url.title.notnull()).sum()
(df_url.description.notnull() & df_url.title.notnull() & df_url.keywords.notnull()).sum()
df_url.notnull().sum()
df_url = df_url[['index', 'description', 'title']]
df_url = df_url.dropna()
df_url.shape
df_url.description = df_url.description.map(lambda x: x.split('。'))
df_url.description = df_url.description.map(lambda x: rawdcrp2tokendcrp(x))
df_url.title = df_url.title.map(lambda x: x.split('。'))
df_url.title = df_url.title.map(lambda x: rawdcrp2tokendcrp(x))

for line in df_url.description.values:
    corpus = corpus + line
for line in df_url.title.values:
    corpus = corpus + line
corpus = []
np.save('corpus.npy', corpus)

In [13]:
corpus = []
for line in train_dealprofile.deal_description.values:
    corpus.extend(line)
for line in train_dealprofile.deal_title.values:
    corpus.extend(line)
for line in train_dealprofile2.deal_description.values:
    corpus.extend(line)
for line in train_dealprofile2.deal_title.values:
    corpus.extend(line)

In [14]:
np.save('corpus_dealonly.npy', corpus)

In [4]:
import pandas as pd
import numpy as np
import gc
import re
import MeCab
from gensim.models import Word2Vec
from gensim.models import TfidfModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format='%(asctime)s:%(levelname)s : %(message)s', level=logging.INFO)

%matplotlib inline

In [5]:
test_cor = np.load('corpus.npy')

In [17]:
import math

In [18]:
last_count = 0
new_count = 0 
for i in range(6):
    new_count = ((df_order.deal_id.value_counts() <= (10* math.pow(10, i))) == True).sum() - last_count
    print(new_count)
    last_count += new_count 

47255
19198
1860
136
9
0


In [2]:
sell_count = df_order.deal_id.value_counts()
sell = pd.DataFrame(sell_count)
sell = sell.rename(columns= {'deal_id':'sell_count'})
sell = sell.reset_index()
sell.columns = ['deal_id','sell_count']

In [3]:
df_dealprofile.shape

(73965, 35)

In [4]:
result = pd.merge(df_dealprofile, sell, how ='inner', on= ['deal_id'])

In [5]:
result.shape

(2162, 36)

In [6]:
result = pd.merge(df_dealprofile, df_order, how ='inner', on= ['deal_id'])

In [15]:
result = pd.merge(df_dealprofile, df_deal, how ='inner', on= ['deal_id'])

In [16]:
result = pd.merge(df_dealprofile2, sell, how ='inner', on= ['deal_id'])

In [19]:
result = pd.merge(df_dealprofile, df_deal, how ='inner', on= ['deal_id'])

In [25]:
result = pd.merge(result, sell, how = 'inner', on=['deal_id'])

In [26]:
result = result[['deal_id','sell_count','deal_category_cd',]]

In [36]:
df_new = pd.read_csv('df_new.csv.gz', sep = ',', compression='gzip')

In [48]:
df_new.deal_description = df_new.deal_description.map(lambda x: eval(x))

In [58]:
result = pd.merge(sell, df_new, how = 'inner', on=['deal_id'])

# session-based embedding

In [96]:
df_order.deal_id = df_order.deal_id.map(lambda x: str(x))

In [98]:
grouped_order = df_order.groupby('account_id').apply(lambda x: ' '.join(list(x['deal_id'])))

In [100]:
Input_Data = pd.DataFrame(grouped_order)
Input_Data = Input_Data.reset_index(drop = True)
Input_Data = Input_Data.iloc[:,0].values
corpus = []
for line in Input_Data:
    words = [x for x in line.split(' ')]
    corpus.append(words)

In [101]:
corpus

[['184562'],
 ['171736',
  '171971',
  '171968',
  '174521',
  '173125',
  '173752',
  '194920',
  '195425',
  '194722',
  '200441',
  '202927',
  '208267',
  '215461',
  '215694',
  '214525',
  '222150',
  '227668',
  '226020',
  '242648',
  '249486',
  '250899',
  '257910',
  '261103',
  '262281'],
 ['169141'],
 ['171971'],
 ['169110', '191989', '188005', '196397', '200441', '200595'],
 ['167508', '168112', '189749', '195808', '215636', '226718', '235506'],
 ['161648',
  '167022',
  '169380',
  '171971',
  '183134',
  '191650',
  '200441',
  '209400',
  '248089',
  '254938'],
 ['163294'],
 ['238897', '269576'],
 ['168268',
  '184118',
  '203410',
  '200441',
  '211004',
  '225317',
  '227034',
  '235436'],
 ['173042'],
 ['171971'],
 ['207457', '209831', '266278'],
 ['235166'],
 ['161622',
  '161647',
  '161648',
  '163692',
  '166821',
  '167408',
  '167340',
  '169098',
  '171084',
  '172389',
  '177905',
  '177384',
  '176785',
  '176789',
  '183467',
  '192248',
  '196658',
  '201

In [102]:
np.save('session_based_corpus.npy', corpus)

In [103]:
Input_Data

array(['184562',
       '171736 171971 171968 174521 173125 173752 194920 195425 194722 200441 202927 208267 215461 215694 214525 222150 227668 226020 242648 249486 250899 257910 261103 262281',
       '169141', ..., '257593', '174613', '258860'], dtype=object)

In [104]:
grouped_order = df_order.groupby('account_id').apply(lambda x: ' '.join(list(x['deal_id'])))
Input_Data = pd.DataFrame(grouped_order)

In [24]:
result[result.deal_category_cd == 'BTY'][['deal_title', 'deal_price']]

,deal_title,deal_price
39,"[[7, 月, 3, 日, GRAND, OPEN, 91, ％, OFF, ハリウッドセレ...",9800
111,"[[88, ％, OFF, 組み合わせ, 自在, 7, メニュー, 半年, パスポート, 5...",29800
174,[[ボツ]],5200
178,[[ボツ]],14800
183,[[ボツ]],1
189,"[[スタッフ, 体験, 取材, 美, 尻, メイク, 2, 回, 券, 心斎橋, 駅, 徒歩...",5400
192,"[[贅沢, 210, 分, 3, 枚, 利用, 可, ハーブ, スチーム, 体, 芯, 温め...",4800
2095,[[ボツ]],5800
2248,"[[新規, 限定, 3, 枚, 利用, 可, 贅沢, 4, メニュー, 白髪染め, 可, 深...",4980
2267,[[ボツ]],5800


In [21]:
df_deal.columns

Index(['deal_id', 'deal_group_id', 'branch_deal_flg', 'shop_id',
       'lx_account_id', 'area_caption', 'deal_type_cd', 'sales_method',
       'deal_category_cd', 'deal_adult_cd', 'deal_status', 'start_tm',
       'end_tm', 'deal_price', 'fixed_price', 'discount_rate',
       'sales_price_name', 'fixed_price_name', 'point_rate', 'point',
       'point_caption', 'link_deal_id', 'link_period', 'link_context',
       'soldout_to_end_flg', 'max_amount', 'min_amount', 'set_amount',
       'max_amount_per_account', 'min_amount_per_account',
       'max_amount_per_group', 'usable_day_visible_flg', 'usable_start_dt',
       'usable_end_dt', 'remind_mail', 'after_purchase_remind_mail',
       'active_flg', 'ordered_flg', 'soldout_flg', 'strategy_flg',
       'account_duplicate_chk_flg', 'template_flg', 'mobile_usable_flg',
       'mobile_ticket_password', 'booking_system_flg', 'np_cart_flg',
       'sales_progress_flg', 'deal_score_boost', 'fixed_score_boost_flg',
       'deal_score_boost_male

In [2]:
df_order.columns

Index(['order_id', 'account_id', 'deal_id', 'branch_deal_id', 'preorder_id',
       'pay_method', 'card_hash', 'order_amount', 'order_value',
       'shipping_cost', 'used_point', 'point_rate', 'gift_id',
       'external_gift_id', 'external_gift_account_id', 'gift_discount',
       'np_payment_fee', 'cod_fee', 'access_id', 'access_password',
       'order_status', 'order_method', 'pay_times', 'transaction_id',
       'transaction_tm', 'refund_value', 'cancel_point', 'pay_env_cd',
       'cart_id', 'del_flg', 'ins_tm', 'upd_tm'],
      dtype='object')

In [17]:
df_order = df_order[['account_id', 'deal_id', 'ins_tm']]

In [18]:
offset = np.zeros(df_order.account_id.nunique()+1, dtype = np.int32)

In [20]:
offset.shape

(341410,)

In [22]:
offset[1:] = df_order.groupby('account_id').size().cumsum()

In [23]:
offset

array([      0,       1,      25, ..., 1682699, 1682700, 1682701],
      dtype=int32)

In [24]:
item_id = df_order.deal_id.unique()

In [25]:
item_id.shape

(68458,)

In [27]:
item2idx = pd.Series(data=np.arange(len(item_id)),index=item_id)

itemmap = pd.DataFrame({'item_ID':item_id,'item_idx':item2idx[item_id].values})
        

In [1]:
itemmap.head(10)

NameError: name 'itemmap' is not defined

In [2]:
df_order = df_order[['account_id', 'deal_id']]
df_order['rating'] = 1

In [3]:
df_order.account_id = df_order.account_id.astype('category').cat.codes.values
df_order.deal_id = df_order.deal_id.astype('category').cat.codes.values

In [4]:
import random

In [5]:
item_list = list(df_order.deal_id.unique())
bought_list = list(df_order[df_order.account_id == 24350].deal_id.values)
negative_list = list(set(item_list)^set(bought_list))

In [6]:
len(item_list), len(bought_list), len(negative_list)

(68458, 23, 68435)

In [15]:
Negative_sample = []
for i in range(len(bought_list)):
        some = random.sample(negative_list, 1)[0]
        Negative_sample.append([24350, some, 0])

In [16]:
Negative_sample

[[24350, 20085, 0],
 [24350, 38848, 0],
 [24350, 59665, 0],
 [24350, 67379, 0],
 [24350, 25218, 0],
 [24350, 57634, 0],
 [24350, 65303, 0],
 [24350, 21299, 0],
 [24350, 47544, 0],
 [24350, 67917, 0],
 [24350, 33538, 0],
 [24350, 24691, 0],
 [24350, 52764, 0],
 [24350, 42455, 0],
 [24350, 28898, 0],
 [24350, 10322, 0],
 [24350, 62573, 0],
 [24350, 11152, 0],
 [24350, 47832, 0],
 [24350, 38964, 0],
 [24350, 18958, 0],
 [24350, 33765, 0],
 [24350, 31818, 0]]

In [2]:
df_order.columns

Index(['order_id', 'account_id', 'deal_id', 'branch_deal_id', 'preorder_id',
       'pay_method', 'card_hash', 'order_amount', 'order_value',
       'shipping_cost', 'used_point', 'point_rate', 'gift_id',
       'external_gift_id', 'external_gift_account_id', 'gift_discount',
       'np_payment_fee', 'cod_fee', 'access_id', 'access_password',
       'order_status', 'order_method', 'pay_times', 'transaction_id',
       'transaction_tm', 'refund_value', 'cancel_point', 'pay_env_cd',
       'cart_id', 'del_flg', 'ins_tm', 'upd_tm'],
      dtype='object')

In [3]:
df_order[['order_id', 'account_id', 'deal_id', 'branch_deal_id', 'preorder_id',
       'pay_method', 'card_hash', 'order_amount', 'order_value',
       'shipping_cost', 'used_point', 'point_rate', 'gift_id',]].values

array([['162624-00385343-1577B9BF03E', 385343, 162624, ..., 0, 1,
        213237.0],
       ['163627-05302679-1577B9C1E7C', 5302679, 163627, ..., 0, 1,
        220243.0],
       ['163689-01544517-1577B9C2DA6', 1544517, 163689, ..., 400, 1, nan],
       ...,
       ['270342-00548325-15ED34D0476', 548325, 270342, ..., 0, 1, nan],
       ['268605-01959766-15ED34D0C45', 1959766, 268605, ..., 0, 1,
        403082.0],
       ['273165-00765865-15ED34D152D', 765865, 273165, ..., 0, 1,
        403082.0]], dtype=object)

In [38]:
df_dealprofile2.ins_tm.tail()

316019    2017-12-21 18:15:38
316020    2017-12-21 18:16:36
316021    2017-12-21 18:17:08
316022    2017-12-21 18:18:50
316023    2017-12-21 18:18:56
Name: ins_tm, dtype: object

In [13]:
df_order = df_order[['account_id', 'deal_id']]

In [18]:
df_order['value_count'] = 1

/home/mmde-lab/anaconda3/envs/whxPyEnv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
df_buy = df_order.groupby(['account_id', 'deal_id']).sum()

In [27]:
df_buy.loc[df_buy.value_count > 10]

,,value_count
account_id,deal_id,
901251,175072,13
1644994,187693,11
7985813,226503,16


In [28]:
df_buy.shape

(1649757, 1)

In [29]:
(df_buy.value_count > 1).sum()

28897

In [30]:
28897 / 1649757

0.017515912949604095

In [3]:
df_dealprofile2.loc[df_dealprofile2.deal_id == 258860].deal_title.values

array(['【プレゼントキャンペーン実施中／スマパス会員限定】世界に誇るブランド和牛「神戸牛」で食卓にごちそうを。コクと旨みの絶妙なバランスと、甘い上質な脂を焼肉で《神戸牛 焼肉用》抽選で50名様にプレゼント'],
      dtype=object)

In [16]:
df_order.loc[df_order.deal_id == 258776].shape

(1790, 32)